## PromptTempalte
- Langchain의 PromptTemplate은 프롬프트를 동적으로 생성할 수 있도록 도와주는 도구입니다
- 변수와 템플릿을 활용해 다양한 입력값에 맞는 프롬프트를 손쉽게 만들 수 있습니다.  
    - `input_variables` : 중괄호 안에 들어갈 변수이름 리스트

공식문서에서 더 다양한 종류의 PromptTemplate들을 확인 할 수 있습니다.
- https://python.langchain.com/api_reference/core/prompts.html

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# OpenAI API 환경변수 값 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"OPENAI_API_KEY가 설정되어 있나요?: {openai_api_key[:10]}...")

LLM 객체 만들기

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.5)

print("모델이 성공적으로 초기화되었습니다!")


`from_template()` 메소드로 `PromptTemplate` 만들기
- 변수로 지정하고 싶은 영역을 `{변수}`로 지정한다.

In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
template = "다음의 동물을 간단한 퀴즈처럼 묘사해주세요, 정답을 말하면 안됩니다! : {input}"

prompt_template = PromptTemplate.from_template(template)

print(prompt_template)

`{변수}`로 지정한 변수에 값을 넣어서 prompt를 완성 할 수 있다.

In [ ]:
prompt = prompt_template.invoke(input="bear")

print(prompt)

In [ ]:
chain = prompt_template | llm

response = chain.invoke(input = "bear")

print(response.content)

`partial_variables`를 사용하여 기본값을 지정할 수 있다.

In [ ]:
template = "{input1}와 {input2}가 싸우면 누가 이길까요?"

prompt_template = PromptTemplate(
    template=template,
    input_variables=["input1"],
    partial_variables={"input2": "bear"},
)

print(prompt_template)

format() : 빈칸({})을 실제 값으로 채워 완성된 프롬프트 문자열을 만드는 함수

In [ ]:
prompt_template.format(input1 = "tiger")

output parser 준비

In [ ]:
from langchain_core.output_parsers import StrOutputParser


output_parser = StrOutputParser()

In [ ]:
chain = prompt_template | llm | output_parser

response = chain.invoke("tiger")

response

## ChatPromptTemplate
- `ChatPromptTemplate`은 여러 역할의 메시지를 순서대로 넣어서 대화 프롬프트를 쉽게 만들 수 있는 도구
- 챗봇처럼 대화가 오갈 때 각 메시지를 따로 템플릿으로 만들고, 한 번에 조합해서 사용할 수 있음.
- 구조
   - 각 메시지는 역할(role)과 내용(content)로 구성됨
   - role의 종류
     - system: 시스템의 기본 지침이나 규칙을 안내할 때 사용
     - human: 사용자의 질문이나 요청을 입력할 때 사용
     - ai: AI의 답변을 입력할 때 사용

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 일본 애니메이션을 좋아하는 사람입니다. 당신은 항상 대답의 끝에 `..랄까나?` 을 붙입니다."),
        ("human", "안녕하세요. AI에 대해 궁금한 점이 있습니다."),
        ("ai", "안녕하세요. 무엇이든 질문해주세요..랄까나?"),
        ("human", "{input}"),
    ]
)

chat_prompt = chat_template.format_messages(input="너는 누구야?")

print(chat_prompt)

In [ ]:
llm.invoke(chat_prompt).content

### MessagePlaceholder
 - `MessagePlaceholder`는 `ChatPromptTemplate`에서 메시지를 동적으로 삽입할 수 있게 해주는 기능
 - 기존 메시지를 재사용하거나 조건에 따라 다른 메시지를 넣을 때 유용함
 - 예를 들어 이전 대화 내용을 기억하고 참조해야 할 때 사용할 수 있음
 - 주요 사용 사례:
   - 대화 기록 유지
   - 동적 시스템 메시지 삽입
   - 조건부 메시지 포함


`HumanMessage, AIMessage, SystemMessage`
- 각각 사람, AI, 시스템의 메시지를 나타내는 객체
- ChatPromptTemplate에서 메시지를 더 명시적이고 타입 안전하게 다룰 수 있음
- 주요 특징:
  - SystemMessage: 시스템의 기본 설정이나 규칙을 정의할 때 사용
  - HumanMessage: 사용자의 입력을 나타낼 때 사용 
  - AIMessage: AI의 응답을 나타낼 때 사용
- 문자열 튜플 대신 이러한 객체를 사용하면 코드의 가독성과 유지보수성이 향상됨


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# 일반 도우미 챗봇 프롬프트 템플릿
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친근한 도우미 챗봇입니다. 이전 대화를 참고하여 답변하세요."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{user_input}")
    ]
)

# 일반적인 챗봇 대화 기록 (짧고 간단)
chat_history = [
    HumanMessage(content="안녕하세요"),
    AIMessage(content="안녕하세요! 무엇을 도와드릴까요?"),
    HumanMessage(content="요즘 야외에서 할 수 있는 새로운 취미를 찾고 있어요"),
    AIMessage(content="새로운 취미를 찾고 계시는군요! 어떤 종류의 활동에 관심이 있으신가요?")
]

# 이전 대화를 참고한 새로운 질문
messages = chat_prompt.format_messages(
    chat_history=chat_history,
    user_input="사람들과 어울릴 수 있으면 좋겠어요" # 사용자입력
)

print(messages)

In [ ]:

llm.invoke(messages).content

In [ ]:

# 이전대화 주석하고 한번 해보기
llm.invoke(messages).content